In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 11.9 MB/s 
     |████████████████████████████████| 182 kB 55.9 MB/s 
     |████████████████████████████████| 7.6 MB 7.7 MB/s 


In [2]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [3]:
# example from https://towardsdatascience.com/clustering-the-20-newsgroups-dataset-with-gpt3-embeddings-10411a9ad150
import pandas as pd
from transformers import GPT2TokenizerFast
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train')
print(list(newsgroups_train.target_names))
print(newsgroups_train.data[:3])
print(newsgroups_train.target[:3])

df_news = pd.DataFrame(list(newsgroups_train.data), columns=['Text'])
df_news["Target"] = list(newsgroups_train.target)

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

df_news['n_tokens'] = df_news.Text.apply(lambda x: len(tokenizer.encode(x)))
df_news = df_news[df_news.n_tokens<2000]
print(len(df_news))

df_news.head(5)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever in

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1119 > 1024). Running this sequence through the model will result in indexing errors


10995


,Text,Target,n_tokens
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,211
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,238
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,506
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,271
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,326


In [4]:
import numpy as np

In [5]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')  # or any other checkpoint
word_embeddings = model.transformer.wte.weight  # Word Token Embeddings 
position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [8]:
import pandas as pd
df = pd.read_csv("/drive/My Drive/Colab Notebooks/data/IST 736 projects/myslice/myslice-statisfied and feedback.csv")

In [10]:
docs = df[~df['myslice.feedback'].isna()]
corpus = df[~df['myslice.feedback'].isna()]

In [11]:
docs = docs['myslice.feedback']
corpus = corpus['myslice.feedback']

In [ ]:
#gpt_embeddings = word_embeddings.encode(corpus)

In [12]:
docs.head()

0    Clearer directions, help contact information, ...
1    Can be difficult to navigate when trying to fi...
2    The platform switch to icons rather than heade...
3    I'd really like the option to set the default ...
4    I do not find Myslice as intuitive to use as i...
Name: myslice.feedback, dtype: object

In [13]:
crTokens = docs.apply(lambda x: len(tokenizer.encode(x)))
# df_news['n_tokens'] = df_news.Text.apply(lambda x: len(tokenizer.encode(x)))

In [14]:
crTokens

0      15
1      13
2      96
3      37
4      19
       ..
475    42
476    42
477    66
478    21
479     8
Name: myslice.feedback, Length: 480, dtype: int64

In [15]:
cr = pd.DataFrame(docs, crTokens)

In [16]:
cr.head()

,myslice.feedback
myslice.feedback,
15,Many of the forms I needed to complete as a ne...
13,It can be difficult to find things. I wish it ...
96,It's very frustrating not to be able to change...
37,Each time I log into MySlice to approve hours ...
19,"Myslice is pretty whack, but I've learned how ..."


In [17]:
cr = pd.DataFrame(docs)

In [18]:
cr.head()

,myslice.feedback
0,"Clearer directions, help contact information, ..."
1,Can be difficult to navigate when trying to fi...
2,The platform switch to icons rather than heade...
3,I'd really like the option to set the default ...
4,I do not find Myslice as intuitive to use as i...


In [19]:
cr = cr.merge(crTokens, left_index = True, right_index = True)

In [20]:
cr.head()

,myslice.feedback_x,myslice.feedback_y
0,"Clearer directions, help contact information, ...",15
1,Can be difficult to navigate when trying to fi...,13
2,The platform switch to icons rather than heade...,96
3,I'd really like the option to set the default ...,37
4,I do not find Myslice as intuitive to use as i...,19


In [21]:
cr = cr.reset_index()

In [22]:
cr.head()

,index,myslice.feedback_x,myslice.feedback_y
0,0,"Clearer directions, help contact information, ...",15
1,1,Can be difficult to navigate when trying to fi...,13
2,2,The platform switch to icons rather than heade...,96
3,3,I'd really like the option to set the default ...,37
4,4,I do not find Myslice as intuitive to use as i...,19


In [23]:
cr = cr.drop(columns = ['index'])
cr = cr.rename({"myslice.feedback_x" : "myslice_feedback"}, axis = 1)
cr = cr.rename({"myslice.feedback_y" : "myslice_feedback_token"}, axis = 1)

In [24]:
cr.head()

,myslice_feedback,myslice_feedback_token
0,"Clearer directions, help contact information, ...",15
1,Can be difficult to navigate when trying to fi...,13
2,The platform switch to icons rather than heade...,96
3,I'd really like the option to set the default ...,37
4,I do not find Myslice as intuitive to use as i...,19


In [25]:
crTokens = crTokens.reset_index()

In [26]:
crTokens = crTokens.drop(columns = ['index'])

In [27]:
crVector = model.transformer.wte.weight[crTokens['myslice.feedback']]

In [28]:
crTokens

,myslice.feedback
0,15
1,13
2,96
3,37
4,19
...,...
475,42
476,42
477,66
478,21


In [29]:
crVector

tensor([[-0.0655, -0.0148,  0.0629,  ..., -0.0407,  0.1432, -0.1243],
        [ 0.0466, -0.0113,  0.0283,  ..., -0.0735,  0.0496,  0.0963],
        [ 0.1482, -0.0096,  0.2772,  ...,  0.0086,  0.0135,  0.1423],
        ...,
        [ 0.1094, -0.1714,  0.1885,  ...,  0.2270,  0.0782,  0.0903],
        [-0.0410, -0.0706,  0.1606,  ...,  0.0348,  0.1737, -0.0178],
        [-0.0797, -0.1248,  0.0563,  ...,  0.0053, -0.0880,  0.1316]],
       grad_fn=<IndexBackward0>)

In [30]:
crVectorDetach = crVector.detach()

In [31]:
crVectorDetach

tensor([[-0.0655, -0.0148,  0.0629,  ..., -0.0407,  0.1432, -0.1243],
        [ 0.0466, -0.0113,  0.0283,  ..., -0.0735,  0.0496,  0.0963],
        [ 0.1482, -0.0096,  0.2772,  ...,  0.0086,  0.0135,  0.1423],
        ...,
        [ 0.1094, -0.1714,  0.1885,  ...,  0.2270,  0.0782,  0.0903],
        [-0.0410, -0.0706,  0.1606,  ...,  0.0348,  0.1737, -0.0178],
        [-0.0797, -0.1248,  0.0563,  ...,  0.0053, -0.0880,  0.1316]])

In [32]:
from sklearn.cluster import KMeans
K =9  # change number of cluster if applicable
gpt_model = KMeans(n_clusters = K, random_state = 2022)
gpt_model.fit(crVectorDetach)
labels = gpt_model.labels_.tolist()  # Save cluster labels in a separate list

In [33]:
def print_docs_closest_to_centroids(model, vec, n):
  K = len(model.cluster_centers_)
  for j in range(K):
    d = model.transform(vec)[:, j] # transform all docs to cluster-distance space
    idx = np.argsort(d)[:n] # find n docs closest to centroid

    c_idx = [m for m, label in enumerate(model.labels_) if label==j]  # find the index of all docs in cluster j   
    print('\n\n======cluster #', j, ', cluster size:', len(c_idx))
    for i in idx:
      if i not in c_idx:
        print('[ this doc is in a different cluster #', model.labels_[i], '>>', corpus[i])
      else:
        print(corpus[i])
  return

In [34]:
def print_cluster_sizes(model):
  cnt_perCluster = {}
  for c in model.labels_:
    cnt_perCluster[c] = cnt_perCluster.get(c, 0) + 1
  print(cnt_perCluster)

In [35]:
print_cluster_sizes(gpt_model)

{2: 31, 0: 103, 5: 71, 1: 132, 4: 79, 3: 23, 6: 18, 7: 13, 8: 10}


In [36]:
print_docs_closest_to_centroids(gpt_model, crVectorDetach, 5)



======cluster # 0 , cluster size: 103
More help by phone versus many back and forth emails would be helpful
I'm not sure who to contact for Myslice help
Usage of MySlice is very confusing and not ease of usage
Training modules/sessions that are easily accessible would be helpful.
MySlice tutorial needed based on various access levels and administrative rights


======cluster # 1 , cluster size: 132
I bit clunky to use. Sometimes login doesn't work (although i think that's mostly a past issue). I wish the Directory were easier to access.
MySlice works when it works. It's not very intuitive. Having to use the navigator for a lot of the functions is very cumbersome and time consuming
Just to need to access with a code every time you log in is annoying, I guess doing it just with your email address it will do, it is enough
Training on the most efficient uses of MySlice would be great.  The new interface can be confusing.  Personally, I would appreciate training on Insights
What used to t

In [37]:
crVectorDetach.size()

torch.Size([480, 768])